In [ ]:
import os
from google.colab import drive
drive.mount("/content/drive")

In [3]:
from torch.nn import functional as F
from imutils import paths
import cv2
import os
import numpy as np
from torch import nn, optim
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torch.utils.tensorboard import SummaryWriter
import torch
from torch.nn import functional as F
import torch.optim as optim
import torch.utils.data
import shutil


writer = SummaryWriter('runs/example')
#=============参数=======================
num_class = 101
epochs = 20
batch_size = 128
device = torch.device( 'cuda' if torch.cuda.is_available() else
                      'cpu')
print(device)
#shutil.rmtree('/kaggle/input/caltech-101/caltech-101/BACKGROUND_Google')

# path = '/content/caltech-101'
path = '/kaggle/input/caltech-101/caltech-101'
image_paths = list(paths.list_images(path))  #返回该目录下所有文件的列表
def data_processor(size=112):
    """
    将文件中的图片读取出来并整理成data和labels 共101类
    :return:
    """
    data = []
    labels = []
    label_name = []
    name2label = {}
    for num, image_path in enumerate(image_paths):
        name = image_path.split(os.path.sep)[-2]  #获取类别名
        if name =='BACKGROUND_Google':continue
        #读取图像并进行处理
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (size, size), interpolation=cv2.INTER_AREA)
        data.append(image)
        label_name.append(name)
    data = np.array(data)
    label_name = list(dict.fromkeys(label_name))  #利用字典进行去重
    label_name = np.array(label_name)
    for num, name in enumerate(label_name):
        name2label[name] = num  #每个类别分配一个标签
    for num, image_path in enumerate(image_paths):
        name = image_path.split(os.path.sep)[-2]
        if name =='BACKGROUND_Google':continue
        labels.append(name2label[image_path.split(os.path.sep)[-2]])
    labels = np.array(labels)
    return data, name2label, labels

#返回标签序号
data, name2label, labels = data_processor()
print(data.shape)
print("===========================")
print(name2label)
print("===========================")
print(labels)

# define transforms
train_transform = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
val_transform = transforms.Compose([transforms.ToPILImage(),transforms.ToTensor(),transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])])
(X, x_val, Y, y_val) = train_test_split(data, labels,test_size=0.1,stratify=labels,random_state=77)
(x_train, x_test, y_train, y_test) = train_test_split(X, Y,test_size=0.111111111,random_state=77)
print(f"x_train examples: {x_train.shape}\nx_test examples: {x_test.shape}\nx_val examples: {x_val.shape}")

#==============================数据加载===============================================
class ImageDataset(Dataset):
    def __init__(self, images, labels=None, transforms=None):
        self.X = images
        self.y = labels
        self.transforms = transforms

    def __len__(self):
        return (len(self.X))

    def __getitem__(self, i):
        data = self.X[i][:]

        if self.transforms:
            data = self.transforms(data)

        if self.y is not None:
            return (data, self.y[i])
        else:
            return data

train_data = ImageDataset(x_train, y_train, train_transform)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data = ImageDataset(x_val, y_val, val_transform)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_data = ImageDataset(x_test, y_test, val_transform)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

#=========================================================
# 网络模型构建
class AlexNet(nn.Module):
    def __init__(self, num_class=101, init_weights=False):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11),
            # nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224] output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(96),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2), #output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(11 * 11 * 256, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_class),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        # print("x.shape", x.shape) #torch.Size([32, 128, 22, 22])
        x = torch.flatten(x, start_dim=1)  #拉成一条
        # print("x.flatten.shape", x.shape)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu') #kaming方法
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)  #正态分布
                nn.init.constant_(m.bias, 0)

#=================构建模型==============================


model = AlexNet(init_weights=True).to(device)   #送入GPU
# model=AlexNet(pretrained=True)
# model.load_state_dict(torch.load('Xlnet.pth'))
model_new = AlexNet().to(device)

criterion = nn.CrossEntropyLoss()  # 交叉熵损失
#optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)  # 随机梯度下降
#optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)
#optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0, rho=0.9, eps=1e-06, weight_decay=0)
optimizer = torch.optim.NAdam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.000, momentum_decay=0.001, foreach=None)

#训练过程
def train(epoch):
    model.train()
    train_loss = 0.0
    train_acc = 0
    step = 1
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        label = label.to(torch.int64)  #类型转换
        optimizer.zero_grad()
        outputs = model(data)
        loss = F.cross_entropy(outputs, label)
        #计算这一个batch的准确率
        acc = (outputs.argmax(dim=1) == label).sum().cpu().item() / 6941
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        train_acc += acc
    #平均数据
    avg_train_acc = train_acc/step
    avg_train_loss = train_loss/step
    writer.add_scalars(
        "Training Loss", {"Training Loss": avg_train_loss},
        epoch
    )
    writer.close()  # 第4步，写入关闭
    return avg_train_acc, avg_train_loss

def val():
    model.eval()
    train_loss = 0.0
    train_acc = 0
    step = 1
    num = 868
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(val_loader):
            data, label = data.to(device), label.to(device)
            label = label.to(torch.int64)  # 类型转换
            # optimizer.zero_grad()
            outputs = model(data)  # torch.Size([32, 101])
            loss = F.cross_entropy(outputs, label)
            # 计算这一个batch的准确率
            acc = (outputs.argmax(dim=1) == label).sum().cpu().item() / num
            train_loss += loss.item()
            train_acc += acc
    #平均数据
    avg_train_acc = train_acc/step
    avg_train_loss = train_loss/step
    return avg_train_acc, avg_train_loss

def test():
    model.eval()
    train_loss = 0.0
    train_acc = 0
    step = 1
    num = 868
    with torch.no_grad():
        for batch_idx, (data, label) in enumerate(test_loader):
            data, label = data.to(device), label.to(device)
            label = label.to(torch.int64)  # 类型转换
            outputs = model(data)
            loss = F.cross_entropy(outputs, label) #交叉熵损失函数
            acc = (outputs.argmax(dim=1) == label).sum().cpu().item() / num
            train_loss += loss.item()
            train_acc += acc
    return train_acc, train_loss

def run():
    print('start training')
    for epoch in range(epochs):
        train_acc, train_loss = train(epoch)
        print("EPOCH [{}/{}] Train acc {:.4f} Train loss {:.4f} ".format(epoch + 1, epochs, train_acc, train_loss))
    torch.save(model.state_dict(),'./model_best.pth')
    val_acc, val_loss = val()
    print("val(): val acc {:.4f} val loss {:.4f} ".format(val_acc, val_loss))
    test_acc, test_loss = test()
    print("test(): test acc {:.4f} test loss {:.4f} ".format(test_acc, test_loss))
    writer.close()

def secondrun():
    print('a new start training')
    epochs = 30
    model = AlexNet().to(device)
    model.load_state_dict(torch.load('Xlnet.pth'))
    optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0005)  # 随机梯度下降
    for epoch in range(epochs):
        train_acc, train_loss = train(epoch)
        print("EPOCH [{}/{}] Train acc {:.4f} Train loss {:.4f} ".format(epoch + 1, epochs, train_acc, train_loss))
    torch.save(model.state_dict(), 'drive/MyDrive/LSH_NET.pth') #保存模型参数
    model_new.load_state_dict(torch.load('drive/MyDrive/LSH_NET.pth'))
    model_new.eval()    # 模型推理时设置
    val_acc, val_loss = val()
    print("val(): val acc {:.4f} val loss {:.4f} ".format(val_acc, val_loss))
    test_acc, test_loss = test()
    print("test(): test acc {:.4f} test loss {:.4f} ".format(test_acc, test_loss))
    writer.close()
run()
#secondrun()

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


cuda
(8677, 112, 112, 3)
{'scorpion': 0, 'hedgehog': 1, 'crocodile_head': 2, 'headphone': 3, 'mandolin': 4, 'ewer': 5, 'stop_sign': 6, 'ant': 7, 'ferry': 8, 'butterfly': 9, 'airplanes': 10, 'ibis': 11, 'lobster': 12, 'buddha': 13, 'yin_yang': 14, 'hawksbill': 15, 'helicopter': 16, 'pigeon': 17, 'snoopy': 18, 'Leopards': 19, 'pagoda': 20, 'Faces': 21, 'cup': 22, 'Faces_easy': 23, 'inline_skate': 24, 'wheelchair': 25, 'wild_cat': 26, 'cougar_body': 27, 'sea_horse': 28, 'emu': 29, 'water_lilly': 30, 'laptop': 31, 'strawberry': 32, 'bass': 33, 'platypus': 34, 'metronome': 35, 'ketch': 36, 'grand_piano': 37, 'lamp': 38, 'gramophone': 39, 'kangaroo': 40, 'starfish': 41, 'dalmatian': 42, 'bonsai': 43, 'wrench': 44, 'flamingo_head': 45, 'umbrella': 46, 'stegosaurus': 47, 'revolver': 48, 'gerenuk': 49, 'beaver': 50, 'barrel': 51, 'anchor': 52, 'saxophone': 53, 'dollar_bill': 54, 'nautilus': 55, 'elephant': 56, 'llama': 57, 'dragonfly': 58, 'cougar_face': 59, 'chandelier': 60, 'Motorbikes': 61, 

In [2]:
!pip install imutils

  Preparing metadata (setup.py) ... done
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25838 sha256=ad290d1e77d14a01a100c70fe525270ecedebf2a88627e51467042f79e2f1cee
  Stored in directory: /root/.cache/pip/wheels/85/cf/3a/e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils


In [ ]:
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);


In [ ]:
!apt install psmisc

In [ ]:
!top

In [ ]:
!ps -aux | grep

In [ ]:
!kill -9 3423

In [1]:
!nvidia-smi

Fri Nov  3 15:47:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%tensorboard --logdir=runs/example

In [ ]:
! pip install tensorboardX


In [5]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# 导入数据集

In [ ]:
!pip install kaggle
import json
token = {"username":"mytoumakazusa","key":"c63ad8ac7f9db5dc57a3a446977a5c5a"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content


In [ ]:
!kaggle datasets download -d imbikramsaha/caltech-101

In [ ]:
!unzip "/content/datasets/imbikramsaha/caltech-101/caltech-101.zip" -d "/content/"


In [ ]:
# ! 表示在 colab jupyter notebook 中执行
!python generate.py -gid 0
